# Axonal spikes

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
import neuron   
import LFPy
from brainsignals.plotting_convention import mark_subplots, simplify_axes
from brainsignals.neural_simulations import return_hallermann_cell
import brainsignals.neural_simulations as ns

np.random.seed(12345)
ns.load_mechs_from_folder(ns.cell_models_folder)

In [ ]:
def insert_current_stimuli(cell):
    stim_params = {'amp': -1.4,
                   'idx': 0,
                   'pptype': "ISyn",
                   'dur': 1e9,
                   'delay': 0}
    synapse = LFPy.StimIntElectrode(cell, **stim_params)
    return synapse, cell

In [ ]:
tstop = 32
dt = 2**-6
cell = return_hallermann_cell(tstop=tstop, dt=dt)

syn, cell = insert_current_stimuli(cell)
cell.simulate(rec_imem=True, rec_vmem=True)

# Time window to extract spike from:
t0 = 18
t1 = 26
t0_idx = np.argmin(np.abs(cell.tvec - t0))
t1_idx = np.argmin(np.abs(cell.tvec - t1))

cell.vmem = cell.vmem[:, t0_idx:t1_idx]
cell.imem = cell.imem[:, t0_idx:t1_idx]
cell.tvec = cell.tvec[t0_idx:t1_idx] - cell.tvec[t0_idx]

In [ ]:
def plot_cell_secs(cell, ax_m, cell_name):
    if "Hallermann" in cell_name:
        plot_idxs = [cell.somaidx[0],
                     #cell.get_closest_idx(z=-1e7),
                     cell.get_closest_idx(z=-500, section="node"),
                     #cell.get_closest_idx(z=0, section="axon"),
                     cell.get_closest_idx(z=600, section="axon")
                     ]

        idx_clr = {idx: ['b', #'cyan',
                         'orange', #'green',
                         'purple'][num]
                   for num, idx in enumerate(plot_idxs)}
    elif "axon" in cell_name:
        plot_idxs = [0, int(cell.totnsegs / 2), cell.totnsegs - 1]
        idx_clr = {idx: ['b', 'cyan', 'orange', 'green', 'purple'][num]
                   for num, idx in enumerate(plot_idxs)}

    else:
        plot_idxs = [cell.somaidx[0],
                     cell.get_closest_idx(z=-1e7),
                     cell.get_closest_idx(z=1e7),]
        idx_clr = {idx: ['b', 'cyan', 'orange', 'green', 'purple'][num]
                   for num, idx in enumerate(plot_idxs)}


    possible_names = ["Myelin", "axon", "Unmyelin", "Node", "node", "my",
                      "hilloc",
                      "hill", "apic", "dend", "soma"]

    sec_clrs = {"Myelin": 'olive',
            "dend": '0.8',
            "soma": '0.8',
            'apic': '0.8',
            "axon": 'purple',
            "node": 'orange',
            "my": '0.3',
            "Unmyelin": 'salmon',
            "Node": 'orange',
            "hilloc": 'lightblue',
            "hill": 'pink',}

    used_clrs = []
    for idx in range(cell.totnsegs):
        sec_name = cell.get_idx_name(idx)[1]
        for ax_name in possible_names:
            if ax_name in sec_name:
                c = sec_clrs[ax_name]
                if not ax_name in used_clrs:
                    used_clrs.append(ax_name)

        ax_m.plot(cell.x[idx], cell.z[idx], '-',
              c=c, clip_on=True, lw=1)

    [ax_m.plot(cell.x[idx].mean(), cell.z[idx].mean(), 'o',
               c=idx_clr[idx], ms=4, zorder=5) for idx in plot_idxs]
    return plot_idxs, idx_clr


print("Plotting")
plt.close("all")
cell_name = "Hallermann"
fig = plt.figure()
fig.subplots_adjust(hspace=0.5, left=0.0, wspace=0.4, right=0.96,
                    top=0.97, bottom=0.1)

ax_m = fig.add_axes([0.01, 0.01, 0.2, 0.99], aspect=1, frameon=False,
                    xticks=[], yticks=[],
                    ylim=[-900, 1200], xlim=[-200, 550])

ax_v = fig.add_axes([0.25, 0.22, 0.18, 0.7], xlabel="time (ms)",
                    ylabel=r"V$_{\rm m}$ (mV)")

ax_eap_norm = fig.add_axes([0.54, 0.22, 0.18, 0.7], xlabel="time (ms)",
                    ylabel=r"norm. V$_{\rm e}$ (close)", 
                          # ylabel="",
                           xlim=[-1, 1])
ax_p2p = fig.add_axes([0.8, 0.22, 0.18, 0.7], xlabel="distance (µm)",
                      ylabel=r"V$_{\rm e}$ amp (µV)")
ax_p2p.grid(True)

plot_idxs, idx_clr = plot_cell_secs(cell, ax_m, cell_name)

num_elecs = 50

lines = []
line_names = []
for idx in plot_idxs:
    tmax_idx = np.argmax(cell.vmem[idx])
    ax_v.plot(cell.tvec, cell.vmem[idx], c=idx_clr[idx])
    ax_v.axvline(cell.tvec[tmax_idx], c=idx_clr[idx], ls=':', lw=0.5)

    sec_name = cell.get_idx_name(idx)[1]
    x_ = cell.x[idx].mean() + cell.d[idx]/2
    y_ = cell.y[idx].mean()
    z_ = cell.z[idx].mean()
    elec_x = np.linspace(x_, x_ + 200, num_elecs)
    elec_y = np.ones(num_elecs) * y_
    elec_z = np.ones(num_elecs) * z_

    elec_parameters = {
        'sigma': 0.3, 
        'x': elec_x,  # electrode requires 1d vector of positions
        'y': elec_y,
        'z': elec_z,
        "method": "root_as_point",
    }
    electrode = LFPy.RecExtElectrode(cell, **elec_parameters)
    electrode.data = electrode.get_transformation_matrix() @ cell.imem * 1e3
    eaps = electrode.data[:, :] - electrode.data[:, 0, None]

    p2p_amps = np.max(eaps, axis=1) - np.min(eaps, axis=1)

    ax_p2p.loglog(elec_x - cell.x[idx].mean(), p2p_amps, c=idx_clr[idx])
    ax_m.plot(elec_x, elec_z, c=idx_clr[idx], ls=':')

    eap_norm = eaps[0] / np.max(np.abs(eaps[0]))
    l, = ax_eap_norm.plot(cell.tvec - cell.tvec[tmax_idx], 
                          eap_norm, c=idx_clr[idx])
    lines.append(l)
    if "soma" in sec_name:
        line_names.append("soma")
    elif "node" in sec_name:
        line_names.append("myelinated axon")
    elif "axon" in sec_name:
        line_names.append("unmyelinated axon")

fig.legend(lines, line_names, frameon=False, ncol=3,
           loc=(0.25, 0.015))
simplify_axes(fig.axes)
mark_subplots(fig.axes[0], "A", ypos=0.95)
mark_subplots(fig.axes[1:], "BCDEF", ypos=1.05)
fig.savefig("test_%s.pdf" % cell_name)



In [ ]:
axon_type = "unmyelinated"
from brainsignals import hallermann_axon_model as ha_ax

if axon_type == "unmyelinated":
    cell_func = ha_ax.return_constructed_unmyelinated_axon
    tstop = 3.5
    cutoff = 1.5
elif axon_type == "myelinated":
    cell_func = ha_ax.return_constructed_myelinated_axon
    tstop = 1.5
    cutoff = 0.0

else:
    raise RuntimeError("Invalid input!")

elec_x = np.array([0])
elec_y = np.array([0])
elec_z = np.zeros(len(elec_x))

# Define electrode parameters
elec_parameters = {
    'sigma': 0.3,   # S/m
    'x': elec_x,  # electrode requires 1d vector of positions
    'y': elec_y,
    'z': elec_z,
    "method": "root_as_point",
}

num_sims = 5
num_splits_clrs = {idx: plt.cm.viridis(idx / 5) for idx in range(5)}

split_angle_fraction = 1
eap_list = []
morph_list = []
tvec = None
for sim_idx in range(num_sims):
    num_splits = sim_idx
    cell = cell_func(dt, tstop, num_splits, split_angle_fraction)

    cell.set_pos(x=0,
                 y=0,
                 z=cell.z[0].mean() - 10)
    print(np.max(cell.z))
    synapse, cell = insert_current_stimuli(cell)
    cell.simulate(rec_vmem=True, rec_imem=True)

    tvec = cell.tvec.copy()
    t0 = np.argmin(np.abs(tvec - cutoff))

    electrode = LFPy.RecExtElectrode(cell, **elec_parameters)
    eaps = electrode.get_transformation_matrix() @ cell.imem * 1e3 * 2

    tvec = tvec[t0:] - tvec[t0]
    eaps = eaps[:, t0:]
    eap_list.append(eaps.copy())
    morph_list.append([cell.x.copy(), cell.y.copy(), cell.z.copy()])

    synapse = None
    cell.__del__()



In [ ]:
print("Plotting")
plt.close("all")
fig = plt.figure(figsize=[6, 2])
fig.subplots_adjust(hspace=0.5, left=0.0, wspace=0.4, right=0.96,
                    top=0.97, bottom=0.1)

ax_m_side = fig.add_axes([0.01, 0.01, 0.27, 0.98], aspect=1, frameon=False,
                    xticks=[], yticks=[], #title="side view",
                   ylim=[-100, 10], xlim=[-32, 37])

ax_eap = fig.add_axes([0.3, 0.17, 0.17, 0.75], xlabel="time (ms)",
                      ylabel=r"V$_{\rm e}$ (µV)")
ax_eap_norm = fig.add_axes([0.55, 0.17, 0.17, 0.75], xlabel="time (ms)",
                      ylabel=r"V$_{\rm e}$ (µV)")

ax_amp = fig.add_axes([0.78, 0.17, 0.17, 0.75], xlabel="bifurcations",
                      ylabel=r"V$_{\rm e}$ amp (µV)", ylim=[0, 12])

p2ps = []

for sim_idx in range(num_sims):
    ax_m_side.plot(elec_x[0], elec_z[0], 'D', c='k', zorder=1000)
    mua_ = eap_list[sim_idx][0]
    ax_eap.plot(tvec, mua_, c=num_splits_clrs[sim_idx], zorder=-sim_idx)
    ax_eap_norm.plot(tvec - tvec[np.argmax(np.abs(mua_))], mua_ / np.max(np.abs(mua_)), 
                     c=num_splits_clrs[sim_idx], zorder=-sim_idx)
    
    p2ps.append(np.max(mua_) - np.min(mua_))
ax_amp.plot(np.arange(num_sims), p2ps, '-o', c='k')

ax_m_side.plot([17, 17], [-45, -65], lw=1, c='k', zorder=1e6)
ax_m_side.text(18, -55, "10\nµm", va="center", ha="left", zorder=1e6)

for sim_idx in range(num_sims):
    x, y, z = morph_list[sim_idx]
    print(sim_idx, np.max(x))
    ax_m_side.plot(x.T, z.T, c=num_splits_clrs[sim_idx], zorder=-sim_idx,
                   rasterized=False)

simplify_axes(fig.axes)
mark_subplots(ax_m_side, "E", xpos=-0.1, ypos=0.97)
mark_subplots([ax_eap, ax_eap_norm, ax_amp], 
              ["F", "G", "H"], xpos=-0.2, ypos=1.06)

fig.savefig("single_axon_branch_MUA_%s.pdf" % axon_type)
